## MOIRAI Usage Example

### Loading MOIRAI Model

In [1]:
# External imports
import os
import sys
import pandas as pd

src_path = os.path.abspath(os.path.join("src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Local imports
from samay.dataset import MoiraiDataset
from samay.model import MoiraiTSModel

# Load the pretrained model
repo = "Salesforce/moirai-moe-1.0-R-small"
config = {
        "context_len": 128,
        "horizon_len": 64,
        "num_layers": 100,
        "model_type": "moirai-moe",
        "model_size": "small"
    }

moirai_model = MoiraiTSModel(repor=repo, config=config)

### Define the datasets

In [2]:
# Config for the electric transformer temperature dataset
data_config = {"name" : "ett",
                "path" : "../src/samay/models/moment/data/ETTh1.csv",
                "date_col" : "date",
                "freq": "h"
            }

df = pd.read_csv(data_config["path"])
df.head()

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000


Here, we have 6 input features (all form `input_ts`) and the target column is the *Oil Temperature* (`OT`)

In [3]:
# Train and test datasets
train_dataset = MoiraiDataset(
    name=data_config['name'],
    mode="train",
    path=data_config['path'],
    datetime_col=data_config['date_col'],
    freq=data_config['freq'],
    context_len=config['context_len'],
    horizon_len=config['horizon_len'],
    normalize=False
)
test_dataset = MoiraiDataset(
    name=data_config['name'],
    mode="test",
    path=data_config['path'],
    datetime_col=data_config['date_col'],
    freq=data_config['freq'],
    context_len=config['context_len'],
    horizon_len=config['horizon_len'],
    normalize=False
)
test_dataset1 = MoiraiDataset(
    name=data_config['name'],
    mode="val",
    path=data_config['path'],
    datetime_col=data_config['date_col'],
    freq=data_config['freq'],
    context_len=config['context_len'],
    horizon_len=config['horizon_len'],
    normalize=False
)

In [4]:
print([x for x in dir(train_dataset) if "__" not in x],end=" ")

['batch_size', 'batchsize', 'boundaries', 'context_len', 'data', 'data_path', 'dataset', 'datetime_col', 'freq', 'get_data_loader', 'horizon_len', 'mode', 'name', 'normalize', 'patch_size', 'preprocess', 'save', 'ts_cols'] 

`data` is of `pd.DataFrame` type and `dataset` is of `gluonts.datatset` type

In [5]:
train_all = [x for x in iter(train_dataset.dataset)]
len(train_all), train_all[0], len(train_all[0]["target"])

(7,
 {'start': Period('2016-07-01 00:00', 'H'),
  'target': array([ 5.82700014,  5.69299984,  5.15700006, ..., -7.50199986,
          5.96099997,  3.68400002]),
  'item_id': 'HUFL'},
 13937)

So, `train_dataset` has time series of all the 7 variables (6 independent and 1 dependent). Each of these has `13936` (`80%` of the data) datapoints. Next, we look at the `test_dataset`

In [6]:
val_all = [x for x in iter(test_dataset1.dataset.dataset)]
len(val_all), val_all[0], len(val_all[0]["target"])

(7,
 {'start': Period('2016-07-01 00:00', 'H'),
  'target': array([ 5.82700014,  5.69299984,  5.15700006, ...,  2.81299996,
          9.24300003, 10.11400032]),
  'item_id': 'HUFL'},
 17420)

In [7]:
test_all = [x for x in iter(test_dataset.dataset)]
len(test_all), test_all[0]

(378,
 ({'start': Period('2016-07-01 00:00', 'H'),
   'target': array([ 5.82700014,  5.69299984,  5.15700006, ..., -7.50199986,
           5.96099997,  3.68400002]),
   'item_id': 'HUFL'},
  {'start': Period('2018-02-01 17:00', 'H'),
   'target': array([ 13.32900047,  15.60599995,  18.68700027,  18.01799965,
           16.94599915,  16.6779995 ,  16.00799942,  15.06999969,
           14.19999981,  14.2670002 ,  14.73600006,  13.86499977,
           12.92700005,  12.72599983,  14.80300045,  13.32900047,
            2.00900006,  -4.95699978,  -5.55900002, -10.44900036,
           -3.54999995,  -2.14299989,   1.47399998,   6.296     ,
           12.99400043,  15.60599995,  17.75      ,  17.28100014,
           17.61599922,  15.8739996 ,  16.87899971,  16.34300041,
           14.46800041,  14.93700027,  13.9989996 ,  13.79800034,
           14.19999981,  13.79800034,  16.34300041,  15.47200012,
            4.62200022,  -4.62200022,  -8.70699978,  -7.76999998,
            0.40200001,  -2.47

There are two parts to each instance (window) in `test_all` - the `input` (context) and the `label` which consists of the true value of the forecast

In [8]:
len(test_all[0][0]["target"]), len(test_all[0][1]["target"])

(13937, 64)

### Evaluate zero-shot forecasting

In [9]:
eval_results, trues, preds, histories = moirai_model.evaluate(test_dataset, metrics=["MSE", "MASE"])
print(eval_results)

{'MSE': 15.595148778108097, 'MASE': 2.092038731177724}


### Finetune MOIRAI on ETT Dataset

In [10]:
finetuned_model = moirai_model.finetune(train_dataset)

NotImplementedError: 